In [13]:
import pandas as pd
import numpy as np
import itertools
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")

In [14]:
# Load dan parsing data
df = pd.read_csv('Stocksastra.csv')
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df.set_index('Tanggal', inplace=True)
df = df.asfreq('B')  # Business day frequency
y = df['Terakhir'].fillna(method='ffill')


In [15]:
p_range = range(0, 3)
d_range = range(0, 2)
q_range = range(0, 3)
P_range = range(0, 3)
D_range = range(0, 2)
Q_range = range(0, 3)
seasonal_period = 6

param_combinations = list(itertools.product(p_range, d_range, q_range))
seasonal_combinations = list(itertools.product(P_range, D_range, Q_range))

best_aic = np.inf
best_order = None
best_seasonal_order = None
best_model = None

for order in param_combinations:
    for seasonal in seasonal_combinations:
        seasonal_order = (seasonal[0], seasonal[1], seasonal[2], seasonal_period)
        try:
            model = SARIMAX(y, order=order, seasonal_order=seasonal_order,
                            enforce_stationarity=False, enforce_invertibility=False)
            result = model.fit(disp=False)
            print(f"✅ Checked order={order}, seasonal={seasonal_order}, AIC={result.aic:.2f}")
            if result.aic < best_aic:
                best_aic = result.aic
                best_order = order
                best_seasonal_order = seasonal_order
                best_model = result
        except Exception as e:
            print(f"❌ Failed order={order}, seasonal={seasonal_order}. Reason: {e}")

print(f"\n✅ Best SARIMA order: {best_order}, seasonal_order: {best_seasonal_order}, AIC={best_aic:.2f}")


✅ Checked order=(0, 0, 0), seasonal=(0, 0, 0, 6), AIC=4281.88
✅ Checked order=(0, 0, 0), seasonal=(0, 0, 1, 6), AIC=3362.87
✅ Checked order=(0, 0, 0), seasonal=(0, 0, 2, 6), AIC=2596.74
✅ Checked order=(0, 0, 0), seasonal=(0, 1, 0, 6), AIC=-263.05
✅ Checked order=(0, 0, 0), seasonal=(0, 1, 1, 6), AIC=-270.80
✅ Checked order=(0, 0, 0), seasonal=(0, 1, 2, 6), AIC=-265.77
✅ Checked order=(0, 0, 0), seasonal=(1, 0, 0, 6), AIC=-260.73
✅ Checked order=(0, 0, 0), seasonal=(1, 0, 1, 6), AIC=-270.76
✅ Checked order=(0, 0, 0), seasonal=(1, 0, 2, 6), AIC=-271.57
✅ Checked order=(0, 0, 0), seasonal=(1, 1, 0, 6), AIC=-271.12
✅ Checked order=(0, 0, 0), seasonal=(1, 1, 1, 6), AIC=-267.72
✅ Checked order=(0, 0, 0), seasonal=(1, 1, 2, 6), AIC=-264.16
✅ Checked order=(0, 0, 0), seasonal=(2, 0, 0, 6), AIC=-272.54
✅ Checked order=(0, 0, 0), seasonal=(2, 0, 1, 6), AIC=-270.55
✅ Checked order=(0, 0, 0), seasonal=(2, 0, 2, 6), AIC=-272.76
✅ Checked order=(0, 0, 0), seasonal=(2, 1, 0, 6), AIC=-267.99
✅ Checke

In [16]:
y_pred_train = best_model.fittedvalues
y_true_train = y[max(best_order[1], best_seasonal_order[1])*seasonal_period:]  # offset untuk d & D

# Sesuaikan panjang prediksi dengan y_true
y_pred_train = y_pred_train[-len(y_true_train):]

mae = mean_absolute_error(y_true_train, y_pred_train)
rmse = mean_squared_error(y_true_train, y_pred_train, squared=False)
mape = np.mean(np.abs((y_true_train - y_pred_train) / y_true_train)) * 100

print(f"\n📊 Evaluation on training data:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"MAPE = {mape:.2f}%")

# ========== 4. FORECASTING ==========

start_date = pd.to_datetime('2024-07-01') 
end_date = pd.to_datetime('2025-01-31')
forecast_steps = len(pd.date_range(start=start_date, end=end_date, freq='B'))

forecast = best_model.forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=start_date, periods=forecast_steps, freq='B')
forecast_df = pd.DataFrame({'Tanggal': forecast_index, 'Forecast_SARIMA': forecast.values})
forecast_df.set_index('Tanggal', inplace=True)

# ========== 5. EXPORT KE CSV ==========

forecast_df.to_csv("forecast_sarima.csv")
print("\n✅ Forecast berhasil disimpan ke 'forecast_sarima.csv'")


📊 Evaluation on training data:
MAE  = 0.0619
RMSE = 0.0873
MAPE = 1.10%

✅ Forecast berhasil disimpan ke 'forecast_sarima.csv'


In [12]:
import itertools
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

warnings.filterwarnings("ignore")

# Rentang parameter
p_range = range(0, 3)
d_range = range(0, 2)
q_range = range(0, 3)
P_range = range(0, 3)
D_range = range(0, 2)
Q_range = range(0, 3)
seasonal_period = 6  # kamu bisa sesuaikan, misalnya 12 jika data musiman tahunan

# Semua kombinasi parameter SARIMA (p,d,q)(P,D,Q,s)
param_combinations = list(itertools.product(p_range, d_range, q_range))
seasonal_combinations = list(itertools.product(P_range, D_range, Q_range))

best_aic = np.inf
best_order = None
best_seasonal_order = None
best_model = None

for order in param_combinations:
    for seasonal in seasonal_combinations:
        seasonal_order = (seasonal[0], seasonal[1], seasonal[2], seasonal_period)
        try:
            model = SARIMAX(data, order=order, seasonal_order=seasonal_order,
                            enforce_stationarity=False, enforce_invertibility=False)
            model_fit = model.fit(disp=False)
            print(f"Checked order={order}, seasonal={seasonal_order}, AIC={model_fit.aic:.2f}")
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_order = order
                best_seasonal_order = seasonal_order
                best_model = model_fit
        except:
            continue

print(f"\n✅ Best SARIMA order: {best_order} seasonal_order: {best_seasonal_order} (AIC={best_aic})")

# Forecast
start_date = pd.to_datetime('2024-07-01')
end_date = pd.to_datetime('2025-01-31')
forecast_steps = len(pd.date_range(start_date, end_date, freq='B'))

sarima_forecast = best_model.forecast(steps=forecast_steps)
sarima_forecast_index = pd.date_range(start=start_date, periods=forecast_steps, freq='B')
sarima_forecast_df = pd.DataFrame({
    'Tanggal': sarima_forecast_index,
    'Forecast': sarima_forecast.values
})




✅ Best SARIMA order: None seasonal_order: None (AIC=inf)


AttributeError: 'NoneType' object has no attribute 'forecast'

In [ ]:

# Ambil data aktual untuk periode yang diforecast
start_date = pd.to_datetime('2024-07-01')
end_date = pd.to_datetime('2025-01-31')
actual = df.loc[start_date:end_date, 'Terakhir']

# Pastikan panjang sama antara prediksi dan aktual
forecast = forecast[:len(actual)]

# MAE
mae = mean_absolute_error(actual, forecast)

# RMSE
rmse = np.sqrt(mean_squared_error(actual, forecast))

# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(actual, forecast)

# Tampilkan hasil
print(f"📊 Evaluasi Model ARIMA:")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"MAPE : {mape:.2f}%")
